In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools 
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import datajoint as dj
from collections import defaultdict
import pycircstat as pycs
from funconnect import morphology, connectomics, ta3, nda
from tqdm import tqdm
import scipy.stats as stats


Connecting celiib@10.28.0.34:3306


In [5]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20


pinky = dj.create_virtual_module('pinky', 'microns_pinky')
ndap100 = dj.create_virtual_module('pinky_nda', 'microns_pinky_nda')
radtune = dj.create_virtual_module('pinky_radtune', 'microns_pinky_radtune')
spattune = dj.create_virtual_module('pinky_spattune', 'microns_pinky_spattune')
pinky_fc = dj.create_virtual_module('pinky_fc', 'microns_pinky_fc')

schema = dj.schema("microns_pinky_radtune")


# Direction Connection Orientation Scatter Plot

In [6]:
"""
Purpose: 8/15/19
Going to get a table of the significant tuned segment combinations with Somas
And then plot the pairs orientation as a scatter plot

Then going to highlight the ones in red that have a direct connection

"""

'\nPurpose: 8/15/19\nGoing to get a table of the significant tuned segment combinations with Somas\nAnd then plot the pairs orientation as a scatter plot\n\nThen going to highlight the ones in red that have a direct connection\n\n'

In [7]:
# Exclude synapses, segments and restrict by functional confidence 

synapse = (pinky.Synapse - pinky.SynapseExclude) & pinky.CurrentSegmentation
segment = (pinky.Segment - pinky.SegmentExclude) & pinky.CurrentSegmentation
soma = (pinky.AllenSoma - pinky.SegmentExclude) & pinky.CurrentSegmentation

soma = soma & "cell_class='excitatory'"

In [8]:
"""
Gets the combination and their unidirection number of synapses between them
"""

# number of direct connections (synapses) for each functional soma pair (both presyn and postsyn are somas with functional traces)

#gets all of the synapses where the presyn and postsyn are both soma cells
fun_fun_syn = synapse & soma.proj(presyn = 'segment_id') & soma.proj(postsyn = 'segment_id') 
#adds up the number of synapses for each soma-soma combination (some wont have a combination because not have synapse between them)
n_syn = dj.U('presyn', 'postsyn').aggr(fun_fun_syn, n_syn = 'count(*)') # soma-soma pairs with at least one synapse
soma_no_id = dj.U("segmentation","segment_id") & soma
#gets all the possible combinations of soma to soma where they aren't the same segment
all_fun_fun = soma_no_id.proj(presyn = 'segment_id') * soma_no_id.proj(postsyn = 'segment_id') & 'presyn != postsyn'
#makes a generic table for all soma combinations that assymes no synapses between them
rel = (dj.U('presyn', 'postsyn') & all_fun_fun).proj('presyn', 'postsyn', n_syn='0')
#combines the base generic 0 synapse table with the one with actual values
#rel is the base table and they are adding the synapse table to it
total_n_syn = rel.aggr(dj.U('n_add') * n_syn.proj(n_add='n_syn'), n_syn_total='n_syn + ifnull(n_add, 0)', keep_all_rows=True)# fill out all zeros

"""
if made n x n combination box with presyns on x and postsyns on y 
a (where presyn < postsyn) is the top right traingle
b (presyn > postsyn) is bottom right triangle

We want every combination of A --> B added together
A (presyn) to B (postsyn) + B (presyn) to A (postsyn)
"""

#combine a-b and b-a connections into one number, and segment_id1 < segment_id2
attrs = ['presyn', 'postsyn', 'n_syn_total']
a = (total_n_syn & 'presyn < postsyn').proj('presyn', 'postsyn', n_syn1 = 'n_syn_total')
b = (total_n_syn & 'presyn > postsyn').proj(n_syn2 = 'n_syn_total', postsyn = 'presyn', presyn = 'postsyn')

"""adds up the non-directional number of synapses for A --> B and B --> A
## *** can use projections to rename primary keys ***
Uses table a as the base table so that is where presyn is less than the postsyn

"""

undirected_n_syn = (a * b).proj(n_syn_total = 'n_syn1 + n_syn2', segment_id1 = 'presyn', segment_id2 = 'postsyn')
undirected_n_syn




segment_id1 segment id unique within each Segmentation,segment_id2 segment id unique within each Segmentation,n_syn_total calculated attribute
648518346341371119,648518346349386137,0
648518346341371119,648518346349470171,0
648518346341371119,648518346349471156,0
648518346341371119,648518346349471500,0
648518346341371119,648518346349471562,0
648518346341371119,648518346349471565,1
648518346341371119,648518346349471910,0
648518346341371119,648518346349472574,0
648518346341371119,648518346349472601,0
648518346341371119,648518346349473044,0


In [9]:
ex_key = dict(segment_id=648518346341351894,segmentation=3)

vonfit_data = (radtune.BestVonFit.Unit & ex_key).fetch1()
vonfit_data

OrderedDict([('segmentation', 3),
             ('vonbestfit_method', 1),
             ('segment_id', 648518346341351894),
             ('von_hash', '3975d5cb0692a185ecb12767aef38fda'),
             ('base', 0.00608233),
             ('thetas', array([0.99976149, 4.14135415])),
             ('amps', array([0.09545269, 0.04130358])),
             ('sharps', array([22.08464569, 22.08464569])),
             ('flat_pred', 0.0174556),
             ('von_pred', 0.0169626),
             ('shuff_pred', 0.0171703),
             ('von_pred_adv', 0.0118989),
             ('von_p_value', 0.128),
             ('confidence', 1.2574478737658663)])

In [10]:
0.99976149*180/np.pi,4.14135415*180/np.pi,0.99976149*180/np.pi + 180

(57.28211389671066, 237.28211426398843, 237.28211389671065)

In [11]:
seg_ids,thetas,amps = (radtune.BestVonFit.Unit & soma.proj()).fetch("segment_id","thetas","amps")

In [12]:
for identity,theta,amp in zip(seg_ids,thetas,amps):
    if amp[0] <= amp[1]:
        print(identity)
        print("Amp less than")
        print(amp)

648518346349490614
Amp less than
[0. 0.]


# Conclusion: shows that need to add more filters to the significant figures

In [13]:
@schema
class BestVonFit(dj.Computed):
    definition = """
    # Mean responses to radially coherent stimuli
    -> pinky.Segmentation
    -> VonBestFitMethod
    ---
    """

    class Unit(dj.Part):
        definition = """
        -> master
        -> pinky.Segment
        ---
        -> VonHash
        base                     : float                # von mises baseline amplitude
        thetas                   : blob                 # von mises gaussian peak angle (descending amp order)
        amps                     : blob                 # von mises gaussian amplitudes (descending amp order)
        sharps                   : blob                 # von mises gaussian sharpness (descending amp order)
        flat_pred                : float                # predictive performance of flat mean
        von_pred                 : float                # predictive performance of von fit on unshuffled data
        shuff_pred               : float                # predictive performance of von fit on shuffled data
        von_pred_adv             : float                # von mises predictive advantage over shuffled data
        von_p_value              : float                # fit component p_value by reshuffle
        confidence               : double               # confidence value that neuron is good match
        """

    @classmethod
    def get_sig_bestvon(cls, fdve_thresh=None, fve_thresh=None,
                        p_thresh=0.002, vonbestfit_method=1,
                        conf_thresh=1, exclude=True):

        von_rest = (f'vonbestfit_method = {vonbestfit_method} and confidence > {conf_thresh}')

        if fdve_thresh:
            von_rest = von_rest + f' and von_pred_adv > {fdve_thresh}'
        if fve_thresh:
            von_rest = von_rest + f' and von_pred > {fve_thresh}'
        if p_thresh:
            von_rest = von_rest + f' and von_p_value < {p_thresh}'
        sig_rel = ((cls.Unit & von_rest) & pinky.CurrentSegmentation)
        if exclude:
            sig_rel = sig_rel - pinky.SegmentExclude
        return sig_rel

In [14]:
sig_relations_best_von = BestVonFit.get_sig_bestvon(conf_thresh=0.5)
sig_relations_best_von

segmentation segmentation id,vonbestfit_method method index,segment_id segment id unique within each Segmentation,von_hash unique hash for combination of sta analysis methods,base von mises baseline amplitude,thetas von mises gaussian peak angle (descending amp order),amps von mises gaussian amplitudes (descending amp order),sharps von mises gaussian sharpness (descending amp order),flat_pred predictive performance of flat mean,von_pred predictive performance of von fit on unshuffled data,shuff_pred predictive performance of von fit on shuffled data,von_pred_adv von mises predictive advantage over shuffled data,von_p_value fit component p_value by reshuffle,confidence confidence value that neuron is good match
3,1,648518346341356241,66dc6420ca132a85ddda25939f084caa,0.00565237,=BLOB=,=BLOB=,=BLOB=,0.0202752,0.0178552,0.0200274,0.107137,0.001,0.957373658926309
3,1,648518346341356256,73dbecb8885beae030be3c263406f897,0.025249,=BLOB=,=BLOB=,=BLOB=,0.0679271,0.063242,0.0669904,0.0551834,0.001,1.7858262119801065
3,1,648518346341359123,8e54b140337daf69dfda4e9fbe14ae7f,0.0297197,=BLOB=,=BLOB=,=BLOB=,0.141441,0.122979,0.139556,0.117201,0.001,3.708831526488791
3,1,648518346341361981,f0c077c6b3ec8074c419a09fbb23388b,0.00287949,=BLOB=,=BLOB=,=BLOB=,0.0234285,0.0217913,0.0229093,0.0477177,0.001,1.4644267171718275
3,1,648518346341362771,a7d6d9bb9c79de84f8792c46b1516c16,0.047481,=BLOB=,=BLOB=,=BLOB=,0.681801,0.587579,0.668815,0.119148,0.001,0.6869009007690088
3,1,648518346341369370,8e4dd43ab0a7846e5013ecc6025e9acc,0.0193596,=BLOB=,=BLOB=,=BLOB=,0.234322,0.210452,0.230954,0.087497,0.001,0.5961253447331519
3,1,648518346341379312,8f3db7b9d61c1c8458fe115a339c7b86,0.000641483,=BLOB=,=BLOB=,=BLOB=,0.0427635,0.0370261,0.0422116,0.12126,0.001,0.5843029464358273
3,1,648518346341380208,74e215bae05c675a123732aa45a53b69,0.00306518,=BLOB=,=BLOB=,=BLOB=,0.0916323,0.0823298,0.0903508,0.0875341,0.001,0.5335169346113676
3,1,648518346341385331,96d93eb2895fe58e287a514e0f83940d,0.00550576,=BLOB=,=BLOB=,=BLOB=,0.10177,0.0808544,0.100497,0.193006,0.001,0.6286509538088302
3,1,648518346341389643,b09b0148ef83ed2d83aa1983c6c919bb,0.00926537,=BLOB=,=BLOB=,=BLOB=,0.046643,0.0414749,0.0460076,0.0971797,0.001,1.3243637575452651


In [15]:
seg_ids,thetas,amps = (sig_relations_best_von & soma.proj()).fetch("segment_id","thetas","amps")
for identity,theta,amp in zip(seg_ids,thetas,amps):
    if amp[0] <= amp[1]:
        print(identity)
        print("Amp less than")
        print(amp)

In [16]:
len(seg_ids)

42

In [17]:
"""This is what the significant function returns"""
radtune.BestVonFit.Unit() & soma.proj() & 'von_p_value <= 0.002' & "confidence>0.5"

segmentation segmentation id,vonbestfit_method method index,segment_id segment id unique within each Segmentation,von_hash unique hash for combination of sta analysis methods,base von mises baseline amplitude,thetas von mises gaussian peak angle (descending amp order),amps von mises gaussian amplitudes (descending amp order),sharps von mises gaussian sharpness (descending amp order),flat_pred predictive performance of flat mean,von_pred predictive performance of von fit on unshuffled data,shuff_pred predictive performance of von fit on shuffled data,von_pred_adv von mises predictive advantage over shuffled data,von_p_value fit component p_value by reshuffle,confidence confidence value that neuron is good match
3,1,648518346349470171,104cb5878f833fe425a0ee7750d69024,0.0338352,=BLOB=,=BLOB=,=BLOB=,1.97013,1.4807,1.94225,0.234276,0.001,3.365524540011297
3,1,648518346349471565,ea7d60f34fe148d38a772c45ffd9d8d2,0.0,=BLOB=,=BLOB=,=BLOB=,13.4957,7.61779,13.3783,0.42684,0.001,3.1366678362873084
3,1,648518346349472574,121f62f7bfd521edb32dcd89bc0e696b,0.00707987,=BLOB=,=BLOB=,=BLOB=,0.0117504,0.0107282,0.0115648,0.0711946,0.001,1.26700343150085
3,1,648518346349473833,8bb981b069c568955229b118039acae2,0.0,=BLOB=,=BLOB=,=BLOB=,6.72941,3.24852,6.65339,0.505969,0.001,2.627829104732558
3,1,648518346349475536,4e1a88968fc9c4fd63b99635861b8291,0.0239916,=BLOB=,=BLOB=,=BLOB=,0.399129,0.344868,0.393501,0.121848,0.001,2.9011142670377916
3,1,648518346349475573,acc70407cc0825b60a383220645cfad3,0.0,=BLOB=,=BLOB=,=BLOB=,0.195326,0.179368,0.192504,0.0672496,0.001,1.4479411512793823
3,1,648518346349482312,179cf4d8a3038c2320b9e000437c78f3,0.0,=BLOB=,=BLOB=,=BLOB=,0.897127,0.813767,0.88305,0.0772273,0.001,2.512158028028598
3,1,648518346349483124,9916df1ccda2f731198b80dc2a43fb43,0.0849652,=BLOB=,=BLOB=,=BLOB=,8.49383,5.56615,8.38582,0.331967,0.001,3.4279893572594107
3,1,648518346349488659,f7978bf93ae7d47941bef9ca9d26145f,0.0160907,=BLOB=,=BLOB=,=BLOB=,0.0216602,0.0201383,0.0213385,0.055407,0.001,0.5864772576750952
3,1,648518346349491811,449763d90b8a0bb06dde845df8f35af3,0.0181213,=BLOB=,=BLOB=,=BLOB=,0.315991,0.240614,0.312157,0.226407,0.001,2.9586540477492376


# Start of making Orientation vs Orientation graph

In [18]:
undirected_n_syn

segment_id1 segment id unique within each Segmentation,segment_id2 segment id unique within each Segmentation,n_syn_total calculated attribute
648518346341371119,648518346349386137,0
648518346341371119,648518346349470171,0
648518346341371119,648518346349471156,0
648518346341371119,648518346349471500,0
648518346341371119,648518346349471562,0
648518346341371119,648518346349471565,1
648518346341371119,648518346349471910,0
648518346341371119,648518346349472574,0
648518346341371119,648518346349472601,0
648518346341371119,648518346349473044,0


In [19]:
"""
Pseudo-code: 
1) Restrict the synapse table by the neurons that have significant directional tuning
2) Pull down the segment_ids and number of synapses for those tables
3) Pull down the thetas and extract the orientation for each of the segments and store in a dictionary
4) Use orientation dictionary to construct parallel array for orientation 
tuples of each combination in restricted synapse table pulled down
5) Find the total number of possibilities for the number of synapses (see if can bin them into groups)
6) do a scatter plot graph with the different bins of the number of synapses

"""

'\nPseudo-code: \n1) Restrict the synapse table by the neurons that have significant directional tuning\n2) Pull down the segment_ids and number of synapses for those tables\n3) Pull down the thetas and extract the orientation for each of the segments and store in a dictionary\n4) Use orientation dictionary to construct parallel array for orientation \ntuples of each combination in restricted synapse table pulled down\n5) Find the total number of possibilities for the number of synapses (see if can bin them into groups)\n6) do a scatter plot graph with the different bins of the number of synapses\n\n'

In [20]:
sig_relations_best_von = BestVonFit.get_sig_bestvon(conf_thresh=0.5)
sig_relations_best_von_soma = sig_relations_best_von & soma.proj()
sig_relations_best_von_soma

segmentation segmentation id,vonbestfit_method method index,segment_id segment id unique within each Segmentation,von_hash unique hash for combination of sta analysis methods,base von mises baseline amplitude,thetas von mises gaussian peak angle (descending amp order),amps von mises gaussian amplitudes (descending amp order),sharps von mises gaussian sharpness (descending amp order),flat_pred predictive performance of flat mean,von_pred predictive performance of von fit on unshuffled data,shuff_pred predictive performance of von fit on shuffled data,von_pred_adv von mises predictive advantage over shuffled data,von_p_value fit component p_value by reshuffle,confidence confidence value that neuron is good match
3,1,648518346349470171,104cb5878f833fe425a0ee7750d69024,0.0338352,=BLOB=,=BLOB=,=BLOB=,1.97013,1.4807,1.94225,0.234276,0.001,3.365524540011297
3,1,648518346349471565,ea7d60f34fe148d38a772c45ffd9d8d2,0.0,=BLOB=,=BLOB=,=BLOB=,13.4957,7.61779,13.3783,0.42684,0.001,3.1366678362873084
3,1,648518346349472574,121f62f7bfd521edb32dcd89bc0e696b,0.00707987,=BLOB=,=BLOB=,=BLOB=,0.0117504,0.0107282,0.0115648,0.0711946,0.001,1.26700343150085
3,1,648518346349473833,8bb981b069c568955229b118039acae2,0.0,=BLOB=,=BLOB=,=BLOB=,6.72941,3.24852,6.65339,0.505969,0.001,2.627829104732558
3,1,648518346349475536,4e1a88968fc9c4fd63b99635861b8291,0.0239916,=BLOB=,=BLOB=,=BLOB=,0.399129,0.344868,0.393501,0.121848,0.001,2.9011142670377916
3,1,648518346349475573,acc70407cc0825b60a383220645cfad3,0.0,=BLOB=,=BLOB=,=BLOB=,0.195326,0.179368,0.192504,0.0672496,0.001,1.4479411512793823
3,1,648518346349482312,179cf4d8a3038c2320b9e000437c78f3,0.0,=BLOB=,=BLOB=,=BLOB=,0.897127,0.813767,0.88305,0.0772273,0.001,2.512158028028598
3,1,648518346349483124,9916df1ccda2f731198b80dc2a43fb43,0.0849652,=BLOB=,=BLOB=,=BLOB=,8.49383,5.56615,8.38582,0.331967,0.001,3.4279893572594107
3,1,648518346349488659,f7978bf93ae7d47941bef9ca9d26145f,0.0160907,=BLOB=,=BLOB=,=BLOB=,0.0216602,0.0201383,0.0213385,0.055407,0.001,0.5864772576750952
3,1,648518346349491811,449763d90b8a0bb06dde845df8f35af3,0.0181213,=BLOB=,=BLOB=,=BLOB=,0.315991,0.240614,0.312157,0.226407,0.001,2.9586540477492376


In [21]:
pinky.SkeletonContact()

segmentation segmentation id,contact_id,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,contact_x,contact_y,contact_z,presyn_length,postsyn_length,n_synapses,synapse_sizes_mean,majority_label,majority_label_ratio
3,0,648518346341351467,648518346341376363,213252,259562,52565,3962.03,3445.86,0,0.0,8,1.0
3,1,648518346341351467,648518346341382069,230254,192358,48611,1786.24,5333.98,0,0.0,8,1.0
3,2,648518346341351467,648518346341382445,207389,223918,58435,10160.4,9471.47,0,0.0,8,1.0
3,3,648518346341351467,648518346341382445,209425,235416,63081,1440.43,1911.07,0,0.0,8,1.0
3,4,648518346341351467,648518346341382445,211683,245770,66945,22180.2,19708.8,0,0.0,8,1.0
3,5,648518346341351467,648518346341385331,225607,267972,76022,6403.25,6135.18,0,0.0,8,1.0
3,6,648518346341351467,648518346341401564,228597,194911,53377,2715.79,2920.31,0,0.0,8,1.0
3,7,648518346341351467,648518346341411405,226404,201911,61696,3026.8,6586.38,1,1507.0,8,1.0
3,8,648518346341351467,648518346344420099,225592,197864,60522,9282.62,7208.36,0,0.0,8,1.0
3,9,648518346341351467,648518346344465416,201048,214398,69602,4269.58,4297.66,0,0.0,8,1.0


In [24]:
synapse_sig_relations = pinky.Synapse & sig_relations_best_von_soma.proj(presyn="segment_id") & sig_relations_best_von_soma.proj(postsyn="segment_id")
synapse_sig_relations

segmentation segmentation id,synapse_id synapse index within the segmentation,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,"presyn_supervoxel supervoxel id unique within each Segmentation, segments can belong to multiple supervoxels","postsyn_supervoxel supervoxel id unique within each Segmentation, segments can belong to multiple supervoxels",synapse_x (EM voxels),synapse_y (EM voxels),synapse_z (EM voxels),presyn_x (EM voxels),presyn_y (EM voxels),presyn_z (EM voxels),postsyn_x (EM voxels),postsyn_y (EM voxels),postsyn_z (EM voxels),size (EM voxels)
3,1383307,648518346349470171,648518346349504065,96034665579700551,96034665579700720,87820,48962,763,87782,49008,761,87842,48994,761,206
3,1751902,648518346349470171,648518346349493653,99979730480029562,99979730480029726,102324,52304,1272,102338,52346,1269,102288,52280,1270,274
3,2777500,648518346349470171,648518346349504065,101391520589961757,101391520589960959,107060,57030,1773,106966,57004,1771,107040,56978,1770,1403
3,2962955,648518346349470171,648518346349500277,101373919813974951,101373919813961923,106585,40075,1471,106522,40022,1471,106522,40114,1471,1041
3,3334717,648518346349470171,648518346349499896,97998414821741019,97998414821741938,94501,42318,1380,94508,42278,1381,94490,42352,1380,218
3,375389,648518346349471565,648518346349499636,90129171447033760,90129171447036110,66122,54172,183,66154,54186,182,66078,54156,183,93
3,1973922,648518346349471565,648518346349483124,86476623884331050,86476623884331016,52866,59536,1080,52814,59562,1074,52786,59510,1076,1009
3,2444878,648518346349471565,648518346349475536,97747713285687443,97747713285685337,93920,70718,901,93874,70702,900,93934,70696,899,602
3,2504303,648518346349471565,648518346349483124,87886197791139555,87886197791140244,57814,61850,1063,57768,61870,1062,57780,61804,1062,668
3,1449898,648518346349475536,648518346349471565,95189132548006585,95189132548005883,84720,48092,499,84740,48052,500,84696,48118,500,216


In [25]:
skeleton_sig_relations = pinky.SkeletonContact() & sig_relations_best_von_soma.proj(postsyn="segment_id")


segmentation segmentation id,contact_id,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,contact_x,contact_y,contact_z,presyn_length,postsyn_length,n_synapses,synapse_sizes_mean,majority_label,majority_label_ratio
3,44,648518346341351467,648518346349483124,208716,217192,57667,1391.82,1724.88,0,0.0,3,1.0
3,45,648518346341351467,648518346349483124,213119,256422,58635,6138.82,6897.33,0,0.0,3,1.0
3,46,648518346341351467,648518346349483124,228543,193425,55616,6232.16,6972.12,0,0.0,3,1.0
3,53,648518346341351467,648518346349488659,197536,224591,63578,7044.13,6429.37,0,0.0,3,1.0
3,58,648518346341351467,648518346349491953,206266,227855,60897,7083.52,6174.52,0,0.0,3,1.0
3,59,648518346341351467,648518346349491953,215970,254985,72701,6042.3,5796.73,0,0.0,3,1.0
3,60,648518346341351467,648518346349491953,221870,263854,75758,5588.42,4852.24,0,0.0,3,1.0
3,62,648518346341351467,648518346349493653,213502,253255,63385,3605.69,3306.07,0,0.0,3,1.0
3,63,648518346341351467,648518346349493653,228221,195180,55486,5653.09,5363.47,0,0.0,3,1.0
3,65,648518346341351467,648518346349494515,219197,261129,74082,6461.4,6510.2,0,0.0,6,1.0


In [27]:
total_postsyn = dj.U("postsyn").aggr(skeleton_sig_relations,total_length="sum(postsyn_length)")
total_postsyn

postsyn segment id unique within each Segmentation,total_length calculated attribute
648518346349470171,135000350.94358063
648518346349471565,249633668.52802277
648518346349472574,124602053.84296417
648518346349473833,31772061.090953827
648518346349475536,87674833.1657486
648518346349475573,3876858.7556877136
648518346349482312,189979039.46297073
648518346349483124,152644801.0540619
648518346349488659,107982911.19549942
648518346349491811,28261718.607271194


In [28]:
total_postsyn & "total_length<100000"

postsyn segment id unique within each Segmentation,total_length calculated attribute


In [29]:
total_data = total_postsyn.fetch(as_dict=True)

In [31]:
contact_length_array = [k["total_length"] for k in total_data]

In [ ]:
"""
Add the contact restriction to only those neurons that have significant contacts 

Pseudocode: 
1) Find the contact length for each of the segment pairs
2) Find the total contact length
3) Filter the pairs by only a certain contact length

"""

In [ ]:
(42*42 - 42)/2

In [ ]:
shared_input_filtered = pinky_fc.SharedInput() & sig_relations_best_von_soma.proj() & sig_relations_best_von_soma.proj(segment_b="segment_id")
shared_input_filtered

In [ ]:
#Step 2: pulling down the filtered segments and the number of synapses
seg_1, seg_2, n_seg_shared = shared_input_filtered.fetch("segment_id","segment_b","n_seg_shared")

In [ ]:
#Step 3:Pull down the thetas and extract the orientation for each of the segments and store in a dictionary
#Step 4: Use orientation dictionary to construct parallel array for orientation  
sig_relations_best_von_soma
seg_ids,thetas,amps = (sig_relations_best_von_soma & soma.proj()).fetch("segment_id","thetas","amps")



In [ ]:
thetas

In [ ]:
#build the lookup dictionary
ori_dict_1 = dict([(seg_id,th[0]*180/np.pi) for seg_id,th in zip(seg_ids,thetas) if th[0]<th[1]])
ori_dict_2 = dict([(seg_id,th[1]*180/np.pi) for seg_id,th in zip(seg_ids,thetas) if th[1]<=th[0]])
ori_dict_2.update(ori_dict_1)
len(ori_dict_2)

In [ ]:
ori_dict_2

In [ ]:
#make sure comes out the same even if just adjust
#build the lookup dictionary
ori_dict_1_check = dict([(seg_id,th[0]*180/np.pi) for seg_id,th in zip(seg_ids,thetas)])
for key,value in ori_dict_1_check.items():
    if value > 180: 
        ori_dict_1_check[key] = value-180
        

#check that they are the same for each key
for key,value in ori_dict_2.items():
    #get the other value in the check
    check_item = ori_dict_1_check[key]
    
    if np.abs(value-check_item)>0.01:
        print(key)
        
"""Conclusion: There is no difference between the two dictionaries built in the two different ways"""

In [ ]:
#build the parallel array
orientation_seg_1 = np.array([ori_dict_2[k] for k in seg_1])
orientation_seg_2 = np.array([ori_dict_2[k] for k in seg_2])
pairs = np.column_stack((orientation_seg_1,orientation_seg_2))
print(pairs)

In [ ]:
#5) Find the total number of possibilities for the number of synapses (see if can bin them into groups)
from collections import Counter
Counter(n_seg_shared)

# Now make the heatmap for the number of shared segments

In [ ]:
flights = sns.load_dataset("flights")
flights

In [ ]:
flights = flights.pivot("month", "year", "passengers")
flights

In [ ]:
ax = sns.heatmap(flights)

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
synapse_data = dict(segment_1_orientation=orientation_seg_1,
                   segment_2_orientation=orientation_seg_2,
                    n_seg_shared=n_seg_shared,
                   )

df = pd.DataFrame.from_dict(synapse_data)
df = df.pivot("segment_1_orientation","segment_2_orientation","n_seg_shared") 
df

In [ ]:
sns.heatmap(df)

In [ ]:
sns.heatmap(df, cmap="YlGnBu")

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
tips = sns.load_dataset("tips")
ax = sns.scatterplot(x="total_bill", y="tip", data=tips)

In [ ]:
ax = sns.scatterplot(x="total_bill", y="tip", hue="time",data=tips)